In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://root:@localhost/pikm')

query = """
SELECT m_kbli.kbli, COUNT(*) as jumlah 
FROM perusahaan
JOIN m_kbli ON perusahaan.kbli_id = m_kbli.id
WHERE perusahaan.kecamatan = 'KUALA'
GROUP BY m_kbli.kbli
"""

try:
   df = pd.read_sql(query, engine)
   print("\nData yang diambil:")
   print(df)

   if not df.empty:
       plt.figure(figsize=(20, 4))
       plt.pie(df['jumlah'], labels=df['kbli'], autopct='%1.1f%%')
       plt.title('Distribusi Jenis Industri di Kecamatan Kuala')
       plt.legend(df['kbli'], 
                 title="Jenis KBLI",
                 loc="center left", 
                 bbox_to_anchor=(1, 0, 0.5, 1))
       
       plt.axis('equal')
       plt.tight_layout()
       plt.show()
       
       print("\nDistribusi Jenis Industri di Kecamatan Kuala:")
       print(df.to_string(index=False))
   else:
       print("\nTidak ada data yang ditemukan untuk kecamatan KUALA")

except Exception as e:
   print(f"Error saat menjalankan query: {e}")
   
finally:
   engine.dispose()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://root:@localhost/pikm')

query = """
SELECT komoditi, SUM(nilaiinvestasi) as total_investasi 
FROM perusahaan 
WHERE kecamatan = 'KUALA' 
AND komoditi IN ('PANGAN', 'PERABOT', 'SANDANG', 'KOSEN', 'KETAMPANG', 
                'BORDIR', 'ALAT PERTANIAN', 'USAHA GARAM', 'AIR MINUM', 
                'BENGKEL RODA 2', 'PENGETAMAN KAYU', 'MENJAHIT')
GROUP BY komoditi
ORDER BY total_investasi DESC
"""

try:
    df = pd.read_sql(query, engine)

    df['percentage'] = df['total_investasi'] / df['total_investasi'].sum() * 100
    df['cumulative_percentage'] = df['percentage'].cumsum()

    plt.figure(figsize=(12, 6))

    ax1 = plt.gca()
    ax2 = ax1.twinx()

    ax1.bar(df['komoditi'], df['total_investasi'], color='skyblue')

    ax2.plot(df['komoditi'], df['cumulative_percentage'], color='red', marker='o', linewidth=2)

    plt.title('Diagram Pareto Nilai Investasi per Komoditi di Kecamatan KUALA', pad=20)
    ax1.set_xlabel('Komoditi')
    ax1.set_ylabel('Nilai Investasi (Rp)')
    ax2.set_ylabel('Persentase Kumulatif (%)')

    plt.xticks(rotation=45, ha='right')

    ax1.grid(True, alpha=0.3)

    plt.tight_layout()

    plt.show()

except Exception as e:
    print(f"Error: {e}")

finally:
    engine.dispose()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://root:@localhost/pikm')

query = """
SELECT *
FROM perusahaan
WHERE (jenisproduk = 'INDUSTRI AIR MINUM DAN AIR MINERAL'
 OR komoditi = 'AIR MINUM')
 AND kecamatan = 'KUALA';
"""

try:
   df = pd.read_sql_query(query, engine)

   industry_counts = df.groupby('kecamatan').size().reset_index(name='jumlah_industri')
   industry_counts = industry_counts.sort_values(by="jumlah_industri", ascending=False)
   industry_counts['cumulative_percentage'] = industry_counts['jumlah_industri'].cumsum() / industry_counts['jumlah_industri'].sum() * 100

   fig, ax = plt.subplots(figsize=(12, 8))

   ax.bar(industry_counts['kecamatan'], industry_counts['jumlah_industri'], color='skyblue')
   ax.set_xlabel('Kecamatan')
   ax.set_ylabel('Jumlah Industri', color='b')
   ax.set_title('Pareto Chart of Industri Air Minum di Kecamatan Kuala')

   ax2 = ax.twinx()
   ax2.plot(industry_counts['kecamatan'], industry_counts['cumulative_percentage'], 
           color='red', marker="D", linestyle='-', linewidth=2)
   ax2.set_ylabel('Cumulative Percentage (%)', color='r')
   ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}%'))

   plt.grid(visible=True, linestyle='--', alpha=0.7)
   plt.tight_layout()
   plt.show()

except Exception as e:
   print(f"Error: {e}")

finally:
   engine.dispose()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://root:@localhost/pikm')

companies = ["MENJAHIT SURYATI", "USAHA PERABOT", "USAHA BORDIR"]

query = """
SELECT perusahaan, nilaiinvestasi, nilaiproduksi 
FROM perusahaan
WHERE perusahaan IN (%s, %s, %s)
"""

try:
   df = pd.read_sql_query(query, engine, params=companies)

   plt.figure(figsize=(12, 8))

   colors = ['#FF6347', '#4682B4', '#32CD32']
   markers = ['o', 's', 'D']

   for i, company in enumerate(companies):
       company_data = df[df['perusahaan'] == company]
       plt.scatter(
           company_data['nilaiinvestasi'],
           company_data['nilaiproduksi'],
           color=colors[i],
           marker=markers[i],
           s=100,
           label=company
       )

   plt.xlabel("Nilai Investasi", fontsize=14)
   plt.ylabel("Nilai Produksi", fontsize=14)
   plt.title("Perbandingan Nilai Investasi vs Nilai Produksi", fontsize=16)
   plt.legend(title="Perusahaan", fontsize=12)

   for i, txt in enumerate(df['perusahaan']):
       plt.annotate(
           txt,
           (df['nilaiinvestasi'].iloc[i], df['nilaiproduksi'].iloc[i]),
           textcoords="offset points", 
           xytext=(5, 5),
           ha='center',
           fontsize=10,
           color='darkblue'
       )

   plt.grid(True, linestyle='--', alpha=0.7)
   plt.tight_layout()
   plt.show()

except Exception as e:
   print(f"Error: {e}")

finally:
   engine.dispose()